<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/answer_Generation_qwen2_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sitting up

In [7]:
!pip install accelerate
!huggingface-cli login
!pip install datasets


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [8]:
from transformers import pipeline
import json
import pandas as pd
from google.colab import files
from datasets import load_dataset
import torch
import os
from IPython.display import display, FileLink

# loading the model

In [11]:
torch.cuda.empty_cache()


In [12]:
# Initialize the pipeline once
generator = pipeline("text-generation", model="Qwen/Qwen2-7B", device_map="auto", torch_dtype="auto")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


# helper functions

In [13]:
def generate_answer(question, num_samples):
    """
    Generates multiple answers for a given question using the model pipeline.
    """
    # Format the prompt
    prompt = f"أجب على السؤال التالي بجملة واحدة موجزة ولكن كاملة.\nQuestion: {question}\nAnswer:"

    # Generate the first 9 answers with temperature=1.0
    outputs_high_temp = generator(
        prompt,
        max_new_tokens=100,
        truncation=True,
        temperature=1.0,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=num_samples - 1
    )

    # Generate the last answer
    outputs_low_temp = generator(
        prompt,
        max_new_tokens=100,
        truncation=True,
        temperature=0.1,
        do_sample=True,
        top_p=0.95,
        num_return_sequences=1
    )

    # Combine the results
    responses = [
        output['generated_text'].split("Answer:")[-1].strip() for output in outputs_high_temp
    ] + [
        outputs_low_temp[0]['generated_text'].split("Answer:")[-1].strip()
    ]

    return responses


# main function to to load questions and gerate answers and save them to a file

---



In [15]:
# Main function
import os
def main():

    # Step 1: Upload and read the file
    file_path = '/content/train-open.json'
    if os.path.exists(file_path):
        print("File found")
        with open(file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)
    else:
      # Load the XQuAD dataset for Arabic (ar)
      xquad_dataset = load_dataset("xquad", "xquad.ar")

      # Load the validation set (the only available split)
      test_set = xquad_dataset['validation']


      # Create an empty list to store the data
      data = []

      # Loop through  and collect data
      for i in range(len(test_set)):  # Loop through all samples
          context = test_set[i]['context']
          question = test_set[i]['question']
          answer = test_set[i]['answers']


          # Append the data as a dictionary to the list
          data.append({
                  "Context": context,
                  "Question": question,
                  "Answer": answer
            })

    # Convert the list of dictionaries to a pandas DataFram
    df = pd.DataFrame(data)

    # Initialize an empty list to store the results
    results = []

    # Step 2:  Generate answers for a sample of questions
    for index, row in df.sample(n=100).iterrows() :
        question = row['Question']
        original_answer = row.get('Answer', "")  # Use empty string if no original answer provided
        question_id = row.get('question_id', index)  # Use index if no question_id provided

        # Generate 10 answers for the 50 questions
        generated_answers = generate_answer(question=question, num_samples=10)

        # Append the results
        results.append({
            "question_id": question_id,
            "question": question,
            "original_answer": original_answer,
            "generated_answers": generated_answers
        })

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(results)
    print(results_df)

#change the file name !!!!!!!!!!!!!!!!!!!!
    filename = "generated_answers_qwen2-7b-xquad.json"
    # Step 3: Save results to a JSON file
    with open(filename, "w", encoding="utf-8") as json_file:
          json.dump(results, json_file, ensure_ascii=False, indent=4)

    print("Generated answers saved to 'generated_answers.json'")
    files.download(filename)

    #what i think next stpes is
    # Step 4: Clustering
    # Step 5: Calculate probability
    # Step 6: Calculate semantic entropy
    # Step 7: show results

# Run the main function
if __name__ == "__main__":
    main()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


    question_id                                           question  \
0           510              ما الاسم الآخر للطريق الحرة يوسيمايت؟   
1           227     ما هي مادّة البث الذي تستعمله سكاي بلس إتس دي؟   
2          1007     ما الدليل الذي تقدمه الجينات المُتَبَرَّع بها؟   
3           796  إلى جانب المدارس الحكومية، ما نوع المدارس المع...   
4          1170          أي شيء كان مفهوم القوة جزءً لا يتجزأ منه؟   
..          ...                                                ...   
95          550           كم شخص مات من الطاعون في باريس عام 1466؟   
96          849      ما هو المقياس الذي يستخدم لقياس قوة الأعاصير؟   
97         1013  ما هو المقسوم الوحيد إلى جانب 1 الذي يمكن أن ي...   
98          816  من هو نائب القنصل الجنوب أفريقي الذي حاصره طلا...   
99          395  ما الذي استوجب القيام بالمعاهدات الأساسية التي...   

                                      original_answer  \
0   {'text': ['الطريق الوطنية 41'], 'answer_start'...   
1   {'text': ['صيغة أم بي إي جي 4'], 'answer_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>